In [ ]:
!pip install ultralytics --quiet

from ultralytics import YOLO
import pandas as pd

# Path to dataset
data_yaml = "/kaggle/input/polyp1/data.yaml"

# Load base model
model = YOLO("yolo11l.pt")

# Train
model.train(
    data=data_yaml,
    epochs=1,
    imgsz=640,
    batch=8,
    name="polyp_detector"
)

# Validate and get metrics
val_results = model.val(data=data_yaml)

# Create dataframe from results
df = pd.DataFrame([{
    "Model": "Polyp Detector",
    "Precision": val_results.results_dict.get('metrics/precision(B)', val_results.box.mp),
    "Recall": val_results.results_dict.get('metrics/recall(B)', val_results.box.mr),
    "mAP50": val_results.results_dict.get('metrics/mAP50(B)', val_results.box.map50),
    "mAP50-95": val_results.results_dict.get('metrics/mAP50-95(B)', val_results.box.map)
}])

print("\n📊 Polyp Model Performance:\n")
print(df)

# Save best weights
!cp "/kaggle/working/runs/detect/polyp_detector/weights/best.pt" "/kaggle/working/polyp_detector_best.pt"
print("\n✅ Model saved as /kaggle/working/polyp_detector_best.pt")


In [ ]:
!pip install ultralytics --quiet

from ultralytics import YOLO
import pandas as pd


data_yaml = "/kaggle/input/stomachulcerds/data.yaml"


model = YOLO("yolo11l.pt")  


model.train(
    data=data_yaml,
    epochs=50,
    imgsz=640,
    batch=8,
    name="ulcer_detector"
)


val_results = model.val()


metrics = val_results.results_dict


df = pd.DataFrame([{
    "Model": "Stomach Ulcer Detector",
    "Precision": metrics['metrics/precision(B)'],
    "Recall": metrics['metrics/recall(B)'],
    "mAP50": metrics['metrics/mAP50(B)'],
    "mAP50-95": metrics['metrics/mAP50-95(B)']
}])

print("\n📊 Stomach Ulcer Model Performance:\n")
print(df)


!cp "/kaggle/working/runs/detect/ulcer_detector/weights/best.pt" "/kaggle/working/ulcer_detector_best.pt"
print("\n✅ Model saved as /kaggle/working/ulcer_detector_best.pt")

In [ ]:
!pip install ultralytics --quiet

from ultralytics import YOLO
import cv2
import os
from matplotlib import pyplot as plt

polyp_model = YOLO("/kaggle/working/polyp_detector_best.pt")
ulcer_model = YOLO("/kaggle/working/ulcer_detector_best.pt")

test_folder = "/kaggle/input/testds1"
image_paths = [os.path.join(test_folder, img) for img in os.listdir(test_folder) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]

print(f"Found {len(image_paths)} test images")

for img_path in image_paths:
    polyp_results = polyp_model(img_path, conf=0.5)
    polyp_img = polyp_results[0].plot()[:, :, ::-1]

    ulcer_results = ulcer_model(img_path, conf=0.5)
    ulcer_img = ulcer_results[0].plot()[:, :, ::-1]

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(polyp_img)
    plt.title(f"Polyp Detector\n{os.path.basename(img_path)}")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(ulcer_img)
    plt.title(f"Ulcer Detector\n{os.path.basename(img_path)}")
    plt.axis("off")

    plt.show()
